# SIMPLE version of the DEMIX notebook
Carlos H Grohmann
version 2022-08-11


# Instructions

This is a simple version of the notebook where we only need to run 3 cells (to run, click in the cell and press shift-enter)

The first cell will download auxiliary data, functions and CSV files from   
GitHub, import python libraries, install the qgrid module and load a selected   CSV and create the ranked dataframe.

### This cell produces a lot of text output. It's safe to clear the output after it's finished 



In [1]:
# # get external files - Friedman tables, custom DEMIX functions and CSV files used for the analysis
# !wget https://github.com/CarlosGrohmann/DEMIX_wine_contest/raw/main/etc_old/Friedmans_tables.zip
# !unzip Friedmans_tables.zip

# !wget https://github.com/CarlosGrohmann/DEMIX_wine_contest/raw/main/demix_wine_functions.py 

# !wget https://github.com/CarlosGrohmann/DEMIX_wine_contest/raw/main/csv_files/demix_20july.csv

# !wget https://github.com/CarlosGrohmann/DEMIX_wine_contest/raw/main/csv_files/hillshade_criterion.csv

# !wget https://github.com/CarlosGrohmann/DEMIX_wine_contest/raw/main/csv_files/toy_example.csv

# # install qgrid, a version that works with Colab
# !pip install git+https://github.com/lukewys/qgrid.git

# imports
import sys,os
import pandas as pd
import numpy as np
from scipy.special import ndtri
import matplotlib.pyplot as plt

sys.path.append('.')
import demix_wine_functions as dw
from IPython.display import display, HTML

# # from google.colab import files
# from google.colab import output
# output.enable_custom_widget_manager()

import qgrid


# 2 - Select CSV file, display the dataframe using qgrid and define filters

## In this cell we can define which CSV file will be opened, by commenting/uncommenting the line with the file name

### You can use the filter controls next to each column name to further select which data you want to be considered  

### Remember:
- ### Set REF_TYPE to DTM **or** DSM 
- ### **DO** filter by LAND_TYPE
- ### Do **NOT** filter the DEM values


In [5]:
# set data directories
root_dir = '.'
tables_dir = f'{root_dir}/Friedmans_tables'

############################################################
########### SELECT CSV FILE HERE ###########################
# comment / uncomment the 
csv_file = 'demix_wine_contest_matrix_18aug2022.csv'
# csv_file = 'demix_wine_contest_matrix_hillshade_criterion.csv'
# csv_file = 'demix_wine_contest_matrix_simple_example.csv'
############################################################

selected_csv_file = [f'{root_dir}/csv_files/{csv_file}']

# make df with one criterion per row
df_criteria = dw.make_criteria_df(selected_csv_file)

# we use this to remove the '-9999' values that come from voids (nodata)
df_criteria.replace({-9999: np.nan}, inplace=True)

# romove some columns which are not important for the wine contest
cols_to_drop = ['REC_ID'] # only in file demix_20july.csv
try:
  df_criteria = df_criteria.drop(cols_to_drop, axis=1)
except:
  pass

# display dataframe with qgrid
grid = qgrid.QGridWidget(df=df_criteria)
display(grid)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# 3 - Run the statistical analysis and check results

### This cell will use the filtered table from the previous cell

### Here you can also select which DEMs will be ranked

In [10]:
# show filters
dw.show_filters(grid)

# get changed (or not) df from qgrid
df_for_ranking = grid.get_changed_df()

# define tolerances for each criterion
tolerance_dict = {
'ELD_AVG':0.5,
'ELD_STD':0.5,
'ELD_RMSE':0.5,
'ELD_MAE':0.5,
'ELD_LE90':0.5,
'RUFD_AVG':0.2,
'RUFD_STD':0.2,
'RUFD_RMSE':0.2,
'RUFD_MAE':0.2,
'RUFD_LE90':0.2,
'SMD_AVG':0.5,
'SMD_STD':0.5,
'SMD_RMSE':0.5,
'SMD_MAE':0.5,
'SMD_LE90':0.5,
'Hillshade':0,
}


# if you don't want to set any tolerance (i.e., set all to Zero),
# uncomment the "tolerance_dict = None" line and it will overwrite the values
# tolerance_dict = None


# make a list of dems being compared, will be needed later
crit_idx  = list(df_criteria.columns).index('CRITERION')
dem_list  = list(df_criteria.columns)[crit_idx+1:]
dem_list.sort() # sort DEMs alphabetically

#################################################################
############### SET DEMs TO BE RANKED HERE ######################
# If you want to change which DEMs will be ranked , do it by
# changing the 'dem_list' variable, removing one or more DEMs

# uncomment the line below and make changes if you want
# dem_list = ['ALOS', 'ASTER', 'COP', 'FABDEM', 'NASA', 'SRTM']
#################################################################

# calculate ranks for metrics in dataframe
df_ranks = dw.make_rank_df(df_for_ranking,dem_list,tolerance_dict,method='average')

#friedman stats
CL = 0.05 # confidence level
dw.friedman_stats(df_ranks,dem_list,tables_dir,cl=CL)
print()
print()

# DEMs ranked and Bonferroni-Dunn test
df_display,df_yn,tie_dict = tables_dems_ranked_bonferroni_dunn(df_ranks,dem_list,alpha=0.95)

print('Ranking Table')
display(HTML(df_display.to_html()))#.replace('\n','<br>'))) # notebook=True
print()
print()

# apply Bonferroni-Dunn test
# df_yn.style.applymap(lambda v: 'opacity: 0%;' if (v==0) else None)
print('Table of Significant Statistical Differences')
display(HTML(df_yn.style.applymap(lambda v: 'opacity: 0%;' if (v==0) else None).to_html()))


No filters applied

Ranking with user-defined tolerances

N = 16 (number of "opinions")
k = 6 (number of DEMs)
CF = 1176.0
chi_r = 32.321
For k=6, CL=0.05, and N=16, the critical value to compare is chi_crit=10.864
And since chi_r (32.321) is greater than chi_crit (10.864)...
Yay!! We can reject the null hipothesis and go to the Post-Hoc analysis!!


Ranking Table


,Sum of ranks,Sum of ranks divided by number of opinions,Not statistically different from
ALOS,32.0,2.000,COP
COP,39.0,2.438,"ALOS,FABDEM,NASA"
FABDEM,53.0,3.312,"COP,NASA,SRTM"
NASA,58.0,3.625,"COP,FABDEM,SRTM"
SRTM,72.0,4.500,"ASTER,FABDEM,NASA"
ASTER,82.0,5.125,SRTM




Table of Significant Statistical Differences


,DEM,ALOS,ASTER,COP,FABDEM,NASA,SRTM
0,ALOS,0,Y,N,Y,Y,Y
1,ASTER,0,0,Y,Y,Y,N
2,COP,0,0,0,N,N,Y
3,FABDEM,0,0,0,0,N,N
4,NASA,0,0,0,0,0,N
5,SRTM,0,0,0,0,0,0
